In [86]:
import pickle
import py
from collections import defaultdict
import pandas as pd
import numpy as np
from utils import calc_scores, _tweet_score, calc_label_corr
import matplotlib.pyplot as plt
import seaborn as sns

In [87]:
from logging import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

In [88]:
dataset_key = 'moral' # or 'bbc' or 'tweet'
dataset_key = 'bbc'

In [89]:
path = py.path.local('results/')
files = path.listdir(fil='cv_results*.pkl')
# files = sorted([fp for fp in files if fp.strpath.endswith('bbc.pkl')]) + sorted([fp for fp in files if not fp.strpath.endswith('bbc.pkl')])
files = [f for f in files if dataset_key in f.strpath and ('mlp' in f.strpath or 'fasttext' in f.strpath) and 'Sandy' not in f.strpath]
files = sorted(files, key=lambda s: s.strpath.split('-')[-1].split('.')[0])
files

[local('/Users/williamferreira/dev/multilabel-stance-detection/results/cv_results_mlp-base_bbc.pkl'),
 local('/Users/williamferreira/dev/multilabel-stance-detection/results/cv_results_mlp-correlation_bbc.pkl'),
 local('/Users/williamferreira/dev/multilabel-stance-detection/results/cv_results_mlp-cross-label-dependency_bbc.pkl'),
 local('/Users/williamferreira/dev/multilabel-stance-detection/results/cv_results_mlp-correlation-global_bbc.pkl'),
 local('/Users/williamferreira/dev/multilabel-stance-detection/results/cv_results_mlp-multitask_bbc.pkl'),
 local('/Users/williamferreira/dev/multilabel-stance-detection/results/cv_results_mlp-powerset_bbc.pkl'),
 local('/Users/williamferreira/dev/multilabel-stance-detection/results/cv_results_fasttext-powerset_bbc.pkl'),
 local('/Users/williamferreira/dev/multilabel-stance-detection/results/cv_results_fasttext-binary-relevance_bbc.pkl')]

In [90]:
all_tweet_scores = defaultdict(defaultdict)
multilabel_scores = {}
for f in files:
    print(f)
    with open(f.strpath, 'rb') as f:
        results = pickle.load(f)
    model_name = results['model_name']
    dataset_name = results['dataset_name']
    print('Dispaying results for model_name: {}, dataset: {}'\
              .format(model_name, dataset_name))
    
    y_true = results['y_test']
    y_pred = results['y_pred']
    print('Test set scores:')
    if results['dataset_name'] == 'bbc' or results['dataset_name'].startswith('moral'):
        scores = calc_scores(y_true, y_pred)
        print(scores)
        multilabel_scores[(model_name, dataset_name)] = scores.loc['OVERALL', :]
    else:
        tweet_score = _tweet_score(y_true.values, y_pred)
        all_tweet_scores[model_name][dataset_name] = tweet_score
        print('Test set macro-averaged F1-score: {}'.format(tweet_score))
    print('Best params: {}\n'.format(','.join(['{}={}'.format(k.split('__')[-1], v) for k, v in results['best_params'].items()])))
    print('Best score: {}'.format(results['best_score']))
    

/Users/williamferreira/dev/multilabel-stance-detection/results/cv_results_mlp-base_bbc.pkl
Dispaying results for model_name: mlp-base, dataset: bbc
Test set scores:
                        accuracy        f1   jaccard precision    recall  \
agreement-disagreement  0.959677         0  0.959677         0         0   
certainty               0.927419      0.25  0.927419       0.6  0.157895   
contrariety             0.834677   0.70073  0.834677  0.695652  0.705882   
hypotheticality         0.935484  0.757576  0.935484  0.925926  0.641026   
necessity                  0.875  0.626506     0.875  0.764706  0.530612   
prediction              0.810484  0.515464  0.810484  0.543478  0.490196   
source-of-knowledge     0.822581  0.614035  0.822581  0.660377   0.57377   
tact-rudeness           0.967742         0  0.967742         0         0   
uncertainty             0.870968       0.6  0.870968  0.666667  0.545455   
volition                 0.96371  0.181818   0.96371         1       0.1   

In [91]:
if dataset_key == 'tweet':
    for m, d in all_tweet_scores.items():
        print(m, np.round(np.mean([list(d.values())]) * 100, 2))
else:
    df = pd.DataFrame(multilabel_scores)
    df.columns = pd.MultiIndex.from_tuples(df.columns)
    df = df.T[['jaccard', 'accuracy', 'f1']].sort_index(level=1)
    print(df['jaccard'].groupby(level=[0, 1]).max().groupby(level=0).mean())

fasttext-binary-relevance     0.397177
fasttext-powerset             0.362030
mlp-base                      0.485685
mlp-correlation               0.468884
mlp-correlation-global        0.447782
mlp-cross-label-dependency    0.513306
mlp-multitask                 0.435148
mlp-powerset                  0.555981
Name: jaccard, dtype: float64


In [94]:
pd.DataFrame(multilabel_scores)

,mlp-base,mlp-correlation,mlp-cross-label-dependency,mlp-correlation-global,mlp-multitask,mlp-powerset,fasttext-powerset,fasttext-binary-relevance
,bbc,bbc,bbc,bbc,bbc,bbc,bbc,bbc
accuracy,0.379032,0.354839,0.294355,0.33871,0.314516,0.439516,0.233871,0.278226
f1,0.424613,0.41818,0.538052,0.368997,0.417435,0.516762,0.315032,0.429068
jaccard,0.485685,0.468884,0.513306,0.447782,0.435148,0.555981,0.36203,0.397177
precision,0.585681,0.603966,0.621563,0.523611,0.578754,0.621604,0.340626,0.563113
recall,0.374484,0.364075,0.547306,0.313725,0.361593,0.465784,0.299244,0.369584
zero_one_loss,0.620968,0.645161,0.705645,0.66129,0.685484,0.560484,0.766129,0.721774
